Two previous parts

In [ ]:
from bs4 import BeautifulSoup
import requests
import html.parser
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import csv                  
import webbrowser
import io
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
cols=['PostalCode','Borough','Neighborhood']
session=requests.Session()
r = session.get(url)
soup=BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', class_='wikitable')
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
output_rows=output_rows[1:]
def stripping(lst):
    return [value.rstrip() for value in lst]
result=list(map(stripping,output_rows))
newresult=list(filter(lambda x: x[1]!='Not assigned',result))
df=pd.DataFrame(newresult, columns=cols)
df.sort_values(by='PostalCode')
coords=pd.read_csv('http://cocl.us/Geospatial_data')
df=df.sort_values(by='PostalCode')
newdf=df.reset_index(drop=True)
newdf['Latitude']=coords['Latitude']
newdf['Longitude']=coords['Longitude']

Coords to center a map

In [ ]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# Plotting map of these neighborhoods

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'], newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare API credentials

In [ ]:
CLIENT_ID = 'VUNTF3ZSXSLZUKZT2E45GMW21D5HJHGK5PVG0Z3MZCTAPWF3' # your Foursquare ID
CLIENT_SECRET = 'GTGDFDX54B2VVS512W5VPQYR5KITNTUNUQ3XUAZIZZZYJVBU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
categories={'Arts & Entertainment': '4d4b7104d754a06370d81259',
            'College & University':'4d4b7105d754a06372d81259',
            'Event':'4d4b7105d754a06373d81259',
            'Food':'4d4b7105d754a06374d81259',
            'Nightlife Spot':'4d4b7105d754a06376d81259',
            'Outdoors & Recreation':'4d4b7105d754a06377d81259',
            'Professional & Other Places':'4d4b7105d754a06375d81259',
            'Residence':'4e67e38e036454776db1fb3a',
            'Travel & Transport':'4d4b7105d754a06379d81259'}



print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# FUnction for exploriпg venues in neighborhoods (actually not in, but around it's coordinates)

def explore_NH(neighborhoods, radius=500,LIMIT=100):
    result=[]
    for i,lst in enumerate(zip(neighborhoods['Neighborhood'],neighborhoods['Latitude'],neighborhoods['Longitude'])):
        names=lst[0]
        lat=lst[1]
        long=lst[2]
        to_append=[names,lat,long]
#         for cats in list(categories.items()):
#             categ=cats[0]
#             catId=cats[1]
        url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
        request_result=requests.get(url).json()
        result.append(request_result)
#         try:
#             to_append.append(len(request_result['response']['venues'])) 
#         except KeyError:
#             print('----------------------------------------------------------------')
#             print(lat, long)
#             print(requests.get(url).json())
#             print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
#         result.append(to_append)
    return result
    


Initial results

In [ ]:
results = explore_NH(newdf)


Checking results

In [ ]:
results[0]['response']['groups'][0]['items'][0]['venue']['categories'][0]['id']

In [ ]:
count=0
for i,qwe in enumerate(results):
    if len(qwe['response']['groups'][0]['items'])>0:
        count+=1
print(count)
print(len(results))

<!-- As you can see there are 100 neighborhoods where are some venues in results.  -->

<!-- Cell below needed to count number of venues and add new column to dataframe -->

In [ ]:
amount=[]
for i in results:
    amount.append(len(i['response']['groups'][0]['items']))

newdf['Venues_count']=amount

print(newdf.head(10))
newdf.tail(10)

I don't want to replicate NY analysis, so here is slightly different analysis

In [ ]:
# Firstly I want to get number of venues in different categories for all neighborhoods

# How?

# I need categories' ids from foursquare.com (https://developer.foursquare.com/docs/build-with-foursquare/categories/)
# I need to scrape them (it's not that easy cus of this page on foursquare.com got interesting structure)
# Next step is to transform this information into useful form - dictionary

def check_hierarchy(to_check):
    if to_check.find_all("li")==[]:
        return 0
    else:
        return 1
    
def to_dict(cats):
    dict_={}
    names=[]
    for zero in cats:
        name_0,cat_0 = zero[0]
#         print(name_0)
        names.append([name_0,cat_0])
        dict_[cat_0]=name_0
        for first in zero[1]:
            if type(first[1]) == list:
                dict_[first[0][1]]=zero[0][1]
                for second in first[1]:
                    if type(second[1]) == list:
                        dict_[second[0][1]] = first[0][1]
                        for third in second[1]:
                            dict_[third[1]] = second[0][1]
                    else:
                        dict_[second[1]] =first[0][1]
                    
            else:
                dict_[first[1]] = zero[0][1]
#     for name,cat in names:
#         print(dict_[cat]==name)
#         if name in dict_.values():
#             print(name, ' OK')
#         else:
#             print(name, ' NOT OK')
    return dict_


def get_venue_categories_dict():
    url = 'https://developer.foursquare.com/docs/build-with-foursquare/categories/'
    session = requests.Session()
    r = session.get(url)
    soup = BeautifulSoup(r.text, 'html')
    main_wrapper = soup.find('ul', class_="VenueCategories__Wrapper-sc-1ysxg0y-0 bYmzDC")
    current = main_wrapper.li
    level_0 = []
    while current != None:
        cat_0 = [str(current.h3.string), str(current.p.string)]
        level_0.append(cat_0)
        to_unpack = list(current.div.children)[-1]
        current_1 = to_unpack.li
        level_1 = []
        while current_1 != None:
            cat_1 = [str(current_1.h3.string), str(current_1.p.string)]
            level_1.append(cat_1)
            if check_hierarchy(current_1):
                current_2 = current_1.li
                level_2=[]
                while current_2 != None:
                    cat_2 = [str(current_2.h3.string), str(current_2.p.string)]
                    level_2.append(cat_2)
                    if check_hierarchy(current_2):
                        level_3 = []
                        current_3 = current_2.li
                        while current_3 != None:
                            cat_3 = [str(current_3.h3.string), str(current_3.p.string)]
                            level_3.append(cat_3)
                            current_3 = current_3.next_sibling
                        level_2[-1] = [level_2[-1], level_3]
                    current_2 = current_2.next_sibling
                level_1[-1] = [level_1[-1], level_2]
                                       
            current_1 = current_1.next_sibling
        level_0[-1] = [level_0[-1], level_1]
        current = current.next_sibling
    return to_dict(level_0)




# Then I need to get main category name using venue's category id

def find_main_category(cat, cat_dict):
#     print('CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT')
#     print(cat)
    if cat_dict[cat] in cat_dict.keys():
        return find_main_category(cat_dict[cat], cat_dict)
    else: return cat_dict[cat]
    
# Now I can get main category name to add columns to dataframe and count number of venues of each category in every NH

def get_categories(data, response, cat_dict):
    for i in range(len(data)):
        ids = []
        for venue in results[i]['response']['groups'][0]['items']:
            ids.append(venue['venue']['categories'][0]['id'])
        dict0={}
        for id_ in ids:
            ID = find_main_category(id_, cat_dict)
            if ID in dict0.keys():
                dict0[ID] += 1
            else:
                dict0[ID] = 1
        for name, val in dict0.items():
            data.loc[i,name] = val
    return data

In [ ]:
newdf.drop(labels='ids', axis=1, inplace=True)
get_categories(newdf, results, get_venue_categories_dict())

In [ ]:
newdf.fillna(0, inplace=True)
newdf

Now I will go right to clustering because frequency analysis here is pretty irrelevant

In [ ]:
from sklearn.cluster import KMeans

kclusters = 6

kmeans = KMeans(n_clusters=kclusters, random_state=13).fit(newdf.iloc[:, -9:])
# check cluster labels generated for each row in the dataframe
kmeans.labels_[20:40]

In [ ]:
# add clustering labels

merged=pd.DataFrame(index=newdf.index)
merged['Latitude']=newdf['Latitude']
merged['Longitude']=newdf['Longitude']
merged['Cluster Labels']=kmeans.labels_
newdf.insert(newdf.shape[1], 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged.head() # check the last columns!



In [ ]:
# create map

import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged.index, merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters